<a href="https://colab.research.google.com/github/hariszaf/metabolic_toy_model/blob/main/Antony2025/reconstructingDraftGSMMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Reconstructing Draft Genome-Scale Metabolic Models**

### **Basic setup**

In case you are working on your local machine, setup **Gurobi** and **COBRApy**. See the [setting up your environment](https://colab.research.google.com/github/hariszaf/metabolic_toy_model/blob/main/Antony2025/preparingYourEnvironment.ipynb). Once ready, activate it: 

```bash
conda activate gsmmWorkshop
```

### **Reconstructing Draft Genome-Scale Metabolic Models**

Draft models are incomplete models containing only genome-based evidence.They are not capable of producing biomass.

To reconstruct draft models, we will use the [ModelSEED](https://academic.oup.com/nar/article/49/D1/D575/5912569?login=true) pipeline.

[see the web interface](https://modelseed.org/)


### **Installing ModelSEED**

Get the ModelSEEDpy repo:

In [ ]:
!git clone https://github.com/ModelSEED/ModelSEEDpy

Cloning into 'ModelSEEDpy'...
remote: Enumerating objects: 4227, done.
remote: Counting objects: 100% (1354/1354), done.
remote: Compressing objects: 100% (263/263), done.
remote: Total 4227 (delta 1188), reused 1100 (delta 1091), pack-reused 2873 (from 2)
Receiving objects: 100% (4227/4227), 8.44 MiB | 4.73 MiB/s, done.
Resolving deltas: 100% (2969/2969), done.
Updating files: 100% (248/248), done.


and then install it:

In [ ]:
!pip install ModelSEEDpy/.

Processing ./ModelSEEDpy
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.6/467.6 kB 8.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.0/26.0 MB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 627.5/627.5 kB 29.8 MB/s eta 0:00:00
  Created wheel for ModelSEEDpy: filename=modelseedpy-0.4.0-py3-none-any.whl size=175143 sha256=e4c741f4c27a6eba27462af27b9338c9ea648e1cf6521c8b2857bd653b87c3e3
  Stored in directory: /tmp/pip-ephem-wheel-cache-5m63anik/wheels/55/a7/fb/a8cad4dc4bbbd40cfb4f4bfce106a58a0b3db07c9501dd91c0
  Created wheel for chemw: filename=ChemW-0.3.5-py3-none-any.whl size=480096 sha256=8efaf71a65708f273022b6762ff9b2e89ac82768fc0d01e26311051c96b3083e
  S

### **Working with genomes**

To reconstruct models, we first need genome sequences. We can use the genomes from EMBL, [ENSEMBL Bacteria](https://bacteria.ensembl.org/index.html), or another public genomic database.

[Here](https://ftp.ensemblgenomes.ebi.ac.uk/pub/bacteria/current/species_EnsemblBacteria.txt) is a list of all the genomes available.

With this list we can perform queries, such retrieve all the genomes from a given taxa.

We also need a package to manipulate NCBI taxonomies: [taxoniq](https://github.com/taxoniq/taxoniq).


### **Parsing taxonomies**

In [6]:
!pip install taxoniq

  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.5/89.5 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 3.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.8/36.8 MB 9.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.2/116.2 MB 5.8 MB/s eta 0:00:0000:0100:01
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 5.4 MB/s eta 0:00:00a 0:00:01


### **Creating a genomes list**

Before reconstructing models, let's perform the following tasks:


1. Make a python dictionary mapping the ENSEMBL genomes to their taxonomies and their fasta file containing genome-enoded proteins.

[genome] = [{taxonomic ranks}, {webpage containing their peptide fasta}]


2. Use our list to find a genome that interests us. For example, we will use a *Bifidobacterium adolescentis* genome;


3. Dowload all genomes belonging to a specific genus. For example, we will use all *Shewanella* genomes;


4. Get one representative genome per phylum.

#### **ENSEMBL Genome Dictionary:**

dowload the genome file

In [ ]:
import urllib.request
urllib.request.urlretrieve("https://ftp.ensemblgenomes.ebi.ac.uk/pub/bacteria/current/species_EnsemblBacteria.txt", "species_EnsemblBacteria.txt")

!ls -la

make a function to generate the dowload url

In [ ]:
import taxoniq
import os

def getProteinFast(l):
    p1 = "https://ftp.ensemblgenomes.ebi.ac.uk/pub/bacteria/current/fasta/"
    p2 = "bacteria" + "_" + l[13].split("_")[1] + "_" + "collection" + "/"
    p3 = l[1] + "/pep/"
    st = l[4]
    while " " in st:
        st = st.replace(" ", "_")
    p4 = l[1][0].upper() + l[1][1:] + "." + st + ".pep.all.fa.gz"

    return p1 + p2 + p3 + p4

genomes = {}
with open('species_EnsemblBacteria.txt') as f:
    f.readline()
    for line in f:
        a = line.strip().split('\t')
        try:
            taxonomy = taxoniq.Taxon(int(a[3]))
            genomes[a[3] + "_" + a[1]] = [{rank.rank.name: rank.scientific_name for rank in taxonomy.ranked_lineage}, getProteinFast(a)]

        except KeyError:
            pass

genomes

#### **Retrieve a genome**

In [ ]:
for genome in genomes:
  if "Bifidobacterium adolescentis" in genomes[genome][0].values():
    print(genome, f" url: {genomes[genome][1]}")

let's pick the first one in the list.

In [ ]:
import gzip
import shutil

url = "https://ftp.ensemblgenomes.ebi.ac.uk/pub/bacteria/current/fasta/bacteria_100_collection/bifidobacterium_adolescentis_atcc_15703_gca_000010425/pep/Bifidobacterium_adolescentis_atcc_15703_gca_000010425.ASM1042v1.pep.all.fa.gz"
gz_id = "Bifidobacterium adolescentis_atcc_15703.fa.gz"
fast_id = "Bifidobacterium adolescentis_atcc_15703.fa"

urllib.request.urlretrieve(url, gz_id)
with gzip.open(gz_id, 'rb') as f_in:
    with open(fast_id, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

os.remove(gz_id)
!ls -la

#### **Download all Shewanella genomes**

In [ ]:
!mkdir shewanella_genomes
#get all Shewanella genomes
root = 'shewanella_genomes'
for genome in genomes:
    if "shewanella" in genome:
        url = genomes[genome][1]
        download_path = os.path.join(root, genome + ".fa.gz")
        extracted_path = os.path.join(root, genome + ".fa")

        urllib.request.urlretrieve(url, download_path)

        with gzip.open(download_path, 'rb') as f_in:
            with open(extracted_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Extracted file saved as {extracted_path}")
        os.remove(download_path)

In [ ]:
!ls shewanella_genomes -la

#### **Get one genome per Phylum**

In [ ]:
!mkdir one_per_phylum

phyla = {}

for genome in genomes:
    if 'phylum' in genomes[genome][0]:#has an annotated phylum
        if genomes[genome][0]['phylum'] not in phyla:
            phyla[genomes[genome][0]['phylum']] = genomes[genome][1]


root = 'one_per_phylum'
for genome in phyla:
    url = phyla[genome]
    download_path = os.path.join(root, genome + ".fa.gz")
    extracted_path = os.path.join(root, genome + ".fa")

    urllib.request.urlretrieve(url, download_path)

    with gzip.open(download_path, 'rb') as f_in:
        with open(extracted_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"Extracted file saved as {extracted_path}")
    os.remove(download_path)

In [ ]:
!ls one_per_phylum -la

ls: cannot access 'one_per_phylum': No such file or directory


### **Annotate Genome**

Before building a draft model, we need to annotate the genome using RAST.

Let's do this with our *Bifidobacterium adolescentis* genome.

In [ ]:
import modelseedpy
from modelseedpy.core.msgenome import MSGenome

from modelseedpy.core.rast_client import RastClient
rast = RastClient()



genome_file = 'Bifidobacterium adolescentis_atcc_15703.fa'



genome = MSGenome.from_fasta(genome_file)

rast.annotate_genome(genome)

for i in genome.features:
    print(i.description)

#### **Reconstruct Draft Model**



To reconstruct the draft genome-scale metabolic model we use the `build_metabolic_model` function of `ModelSEEDpy`. The input to the function is a RAST annotated genome.

In [ ]:
from modelseedpy import MSBuilder

model_id = 'Bifidobacterium adolescentis_atcc_15703'

base_model = MSBuilder.build_metabolic_model(model_id = model_id,
                                             genome   = genome,
                                             index    = "0",
                                             classic_biomass = True,
                                             gapfill_model   = False,
                                             gapfill_media   = None,
                                             annotate_with_rast = True,
                                             allow_all_non_grp_reactions = True
                                            )

We can see the model is a draft and does not produce biomass.

In [ ]:
base_model.optimize()

We save the draft model

In [ ]:
model_name = "Bifidobacterium adolescentis_atcc_15703.sbml"
cobra.io.write_sbml_model(cobra_model = base_model, filename = model_name)

!ls -la

### **Batch reconstruction**

Let's reconstruct one draft model per phylum. First we make a function to reconstruct dract models.

In [ ]:
def reconstruct_draft_model(model_id, input_protein_fasta, output_model_sbml):
    genome = MSGenome.from_fasta(input_protein_fasta, split = ' ')
    rast.annotate_genome(genome)

    base_model = MSBuilder.build_metabolic_model(model_id = model_id,
                                             genome   = genome,
                                             index    = "0",
                                             classic_biomass = True,
                                             gapfill_model   = False,
                                             gapfill_media   = None,
                                             annotate_with_rast = True,
                                             allow_all_non_grp_reactions = True
                                            )

    cobra.io.write_sbml_model(cobra_model = base_model, filename = output_model_sbml)

    return base_model


Now lets run the function for all the genomes in the `one_per_phylum` folder and write the outputs to the folde `one_per_phylum_models`.

In [ ]:
!mkdir shewanella_models

rast = RastClient()

root = "one_per_phylum"
genomes = os.listdir(root)


for name in genomes:
    if ".fa" in name:
        model_id = name.replace(".fa", "")
        model = reconstruct_draft_model(model_id, os.path.join('one_per_phylum', name), os.path.join('one_per_phylum_models', model_id + ".sbml"))
        print(f"Reconstructed {model_id}")



In [ ]:
!ls shewanella_models -la

### **What if we don't have a protein fasta?**

We can use [`Pyrodigal`](https://joss.theoj.org/papers/10.21105/joss.04296) to predict open reading frames and make a protein multifasta for our DNA sequence.

In [ ]:
!pip install pyrodigal
!pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 39.8 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/google/colab/_pip.py:86: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.11/dist-packages/biopython-1.85.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):


In [ ]:
url = 'https://www.ebi.ac.uk/ena/browser/api/fasta/OZ061323?download=true'

download_path = 'OZ061323.fa'

urllib.request.urlretrieve(url, download_path)

!ls -la

total 11224
drwxr-xr-x  1 root root    4096 Feb 24 16:58  .
drwxr-xr-x  1 root root    4096 Feb 24 12:26  ..
-rw-r--r--  1 root root 1011415 Feb 24 15:46 'Bifidobacterium adolescentis_atcc_15703.fa'
-rw-r--r--  1 root root 1871507 Feb 24 16:19 'Bifidobacterium adolescentis_atcc_15703.sbml'
drwxr-xr-x  4 root root    4096 Feb 20 14:24  .config
drwxr-xr-x  8 root root    4096 Feb 24 14:59  DNNGIOR
drwxr-xr-x  2 root root    4096 Feb 24 14:58  licenses
drwxr-xr-x 10 root root    4096 Feb 24 15:04  ModelSEEDpy
drwxr-xr-x  2 root root    4096 Feb 24 16:01  one_per_phylum
drwxr-xr-x  2 root root    4096 Feb 24 16:51  one_per_phylum_models
-rw-r--r--  1 root root 2097404 Feb 24 17:00  OZ061323.fa
drwxr-xr-x  1 root root    4096 Feb 20 14:24  sample_data
drwxr-xr-x  2 root root    4096 Feb 24 15:54  shewanella_genomes
-rw-r--r--  1 root root 6465180 Feb 24 15:34  species_EnsemblBacteria.txt


In [ ]:
from Bio import SeqIO
import pyrodigal

record = SeqIO.read("OZ061323.fa", "fasta")
dna_seq = str(record.seq)  # Convert the sequence to a plain string

print(f"sequence legth: {len(dna_seq)}")

gene_finder = pyrodigal.GeneFinder()

gene_finder.train(dna_seq)

genes = gene_finder.find_genes(dna_seq)

with open("OZ061323.pep.faa", "w") as f:
    genes.write_translations(f, sequence_id="seqXYZ")

sequence legth: 2062891


In [ ]:
!head OZ061323.pep.faa

>seqXYZ_1 # 1 # 1347 # 1 # ID=1_1;partial=10;start_type=Edge;rbs_motif=None;rbs_spacer=None;gc_cont=0.422
MPDLQELWNYLREEFQNDLTPVGFNAWIKTAKPLAFRANEILIEVPSPLHKEYWESNLAT
KVVEGAYEFAEIELTPIFLLPTEAEQLQAEKPAEERSLTKAETPTFLRETHLNSKYTFDT
FVTGKGNQMAHAAALVVSEEPGVLYNPLFLYGGVGLGKTHLMQAIGHQLLLSKPDTNVKY
VTSEAFANDFINSIQTKNQEKFRQEYRNVDLLLVDDIQFFADKEGTQEEFFHTFNDLYND
KKQIVLTSDRLPNEIPKLQERLVSRFKWGLSVDITPPDLETRIAILRNKADTERLEIPED
TLSYIAGQIDSNVRELEGSLVRVQAYATMQNAEITTSLAADALKGLKLNGKSSQLSIAKI
QSVVAKYYSLSITDLKGRKRVKEIVLPRQIAMYLAREMTDSSLPKIGQEFGGKDHTTVMH
AHERISQALTSDQNLKDAILDLKNTLKG*
>seqXYZ_2 # 1530 # 2669 # 1 # ID=1_2;partial=00;start_type=ATG;rbs_motif=GGAGG;rbs_spacer=5-10bp;gc_cont=0.396


In [ ]:
model_id = 'OZ061323'

model = reconstruct_draft_model(model_id, 'OZ061323.pep.faa', 'OZ061323.sbml')

In [ ]:
ls -la

total 13584
drwxr-xr-x  1 root root    4096 Feb 24 17:05  ./
drwxr-xr-x  1 root root    4096 Feb 24 12:26  ../
-rw-r--r--  1 root root 1011415 Feb 24 15:46 'Bifidobacterium adolescentis_atcc_15703.fa'
-rw-r--r--  1 root root 1871507 Feb 24 16:19 'Bifidobacterium adolescentis_atcc_15703.sbml'
drwxr-xr-x  4 root root    4096 Feb 20 14:24  .config/
drwxr-xr-x  8 root root    4096 Feb 24 14:59  DNNGIOR/
drwxr-xr-x  2 root root    4096 Feb 24 14:58  licenses/
drwxr-xr-x 10 root root    4096 Feb 24 15:04  ModelSEEDpy/
drwxr-xr-x  2 root root    4096 Feb 24 16:01  one_per_phylum/
drwxr-xr-x  2 root root    4096 Feb 24 16:51  one_per_phylum_models/
-rw-r--r--  1 root root 2097404 Feb 24 17:00  OZ061323.fa
-rw-r--r--  1 root root  868702 Feb 24 17:03  OZ061323.pep.faa
-rw-r--r--  1 root root 1540747 Feb 24 17:05  OZ061323.sbml
drwxr-xr-x  1 root root    4096 Feb 20 14:24  sample_data/
drwxr-xr-x  2 root root    4096 Feb 24 15:54  shewanella_genomes/
-rw-r--r--  1 root root 6465180 Feb 24 15:34 

In [ ]:
model.optimize()

,fluxes,reduced_costs
rxn00351_c0,0.0,0.0
rxn00836_c0,0.0,0.0
rxn10298_c0,0.0,-0.0
rxn00364_c0,0.0,-0.0
rxn03408_c0,0.0,-0.0
...,...,...
EX_cpd00011_e0,0.0,-0.0
EX_cpd03453_e0,0.0,-0.0
EX_cpd03726_e0,0.0,-0.0
bio1,0.0,0.0
